# TensorBoard: Visualizing Learning

Source: https://www.tensorflow.org/get_started/summaries_and_tensorboard

The computations you'll use TensorFlow for can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, TensorFlow includes a suite of visualization tools called **TensorBoard**. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it.

This part of the tutorial is intended to get you started with simple TensorBoard usage. There are other resources available as well! The [TensorBoard's GitHub](https://github.com/tensorflow/tensorboard) has a lot more information on TensorBoard usage, including tips & tricks, and debugging information.

We'll use the code from the linear regression example and visualize what happens during training using TensorBoard.

In [1]:
from os.path import join

import numpy as np
import tensorflow as tf

As before, we create training and testing data according to the linear model
$y = \text{intercept} + \text{slope} + \text{noise}$.

In [2]:
def make_noisy_data(m=0.1, b=0.3, n=100):
    x = np.random.rand(n).astype(np.float32)
    noise = np.random.normal(scale=0.01, size=len(x))
    y = m * x + b + noise
    return x, y

In [3]:
x_train, y_train = make_noisy_data()
x_test, y_test = make_noisy_data()

Create a new ``tf.Session``.

In [4]:
tf.reset_default_graph()
sess = tf.Session()

## Name scoping and nodes

Typical TensorFlow graphs can have many thousands of nodes – far too many to see easily all at once, or even to lay out using standard graph tools. To simplify, variable names can be scoped and the visualization uses this information to define a hierarchy on the nodes in the graph. By default, only the top of this hierarchy is shown. Here is an example that defines two placeholders under the ``input`` name scope using ``tf.name_scope``:

In [5]:
with tf.name_scope('input'):
    x_placeholder = tf.placeholder(shape=[None], dtype=tf.float32, name='x-input')
    y_placeholder = tf.placeholder(shape=[None], dtype=tf.float32, name='y-input')

We do the same to define the linear model in the ``model`` scope:

In [6]:
with tf.name_scope('model'):
    slope = tf.Variable(0.0, name='slope')
    intercept = tf.Variable(0.0, name='intercept')
    y = slope * x_placeholder + intercept

Next, we define a loss function (here, squared error) and an optimizer (here, gradient descent).

In [7]:
LEARNING_RATE = 0.5

with tf.name_scope('training'):
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(y - y_placeholder))
    with tf.name_scope('optimizer'):
        train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss) 

At this point, our graph is complete. We're now ready to initialize variables.

In [8]:
sess.run(tf.global_variables_initializer())

## Serializing the data

TensorBoard operates by reading TensorFlow events files, which contain summary data that you can generate when running TensorFlow. Here's the general lifecycle for summary data within TensorBoard.

First, create the TensorFlow graph that you'd like to collect summary data from, and decide which nodes you would like to annotate with [summary operations](https://www.tensorflow.org/api_guides/python/summary).

We already created the TensorFlow graph and would like to collect summary data for the loss, slope, and intercept during training. This can be done by attaching nodes, such as ``tf.summary.scalar`` to the respective nodes in our graph.

In [9]:
# Attach summaries to Tensors (for TensorBoard visualization)
with tf.name_scope('summaries'):
    tf.summary.scalar('slope', slope)
    tf.summary.scalar('intercept', intercept)
    tf.summary.scalar('loss', loss)

# This op will calculate our summary data when run
summary_op = tf.summary.merge_all()

Operations in TensorFlow don't do anything until you run them, or an op that depends on their output. And the summary nodes that we've just created are peripheral to your graph: none of the ops you are currently running depend on them. So, to generate summaries, we need to run all of these summary nodes. Managing them by hand would be tedious, so use ``tf.summary.merge_all`` to combine them into a single op that generates all the summary data.

Then, you can just run the merged summary op, which will generate a ``Summary`` object with all of your summary data at a given step. Finally, to write this summary data to disk, pass the summary to a ``tf.summary.FileWriter``.

In [10]:
# This is a directory we'll use to store information
# about the graph to later visualize in TensorBoard.
# By default, it will be created in the same directory
# as this notebook. 

# Be sure to delete the contents of this directory before
# running the script.
LOGDIR = 'graphs'

writer = tf.summary.FileWriter(join(LOGDIR, '02_linear_regression'))
writer.add_graph(sess.graph)

The ``FileWriter`` takes a logdir in its constructor - this logdir is quite important, it's the directory where all of the events will be written out.

Now that you've modified your graph and have a ``FileWriter``, you're ready to start training! We run the merged summary op every single step during training.

In [11]:
TRAIN_STEPS = 201

for step in range(TRAIN_STEPS):
    # Session will run two ops:
    # - summary_op prepares summary data we'll write to disk in a moment
    # - train_step will use the optimizer to adjust our variables to reduce loss
    summary_result, _ = sess.run([summary_op, train_step], 
                                  feed_dict={x_placeholder: x_train, 
                                             y_placeholder: y_train})
    # write the summary data to disk
    writer.add_summary(summary_result, step)

# close the writer when we're finished using it
writer.close()

## Launching TensorBoard

To start TensorBoard, run this command from a terminal:

```
tensorboard --logdir=./graphs
```
**Note**: you should run this from the same directory that contains this notebook
or, provide absolute path to the 'graphs' directory.

To open TensorBoard, point your browser to http://localhost:6006
Then, click on the tabs for 'scalars', 'distributions', 'histograms', and 'graphs'
to learn more.

If you run into trouble, delete ``LOGDIR`` (to clear information from previous runs),
then re-run this script, and restart TensorBoard.

For more usage information on TensorBoard in general, see the [TensorBoard's GitHub](https://github.com/tensorflow/tensorboard).